In [ ]:
# THIS IS THE ORIGINAL ONE THAT IS NOW GIVING ERROS

In [ ]:
from torch._functorch.aot_autograd import aot_module_simplified
from torch._decomp import core_aten_decompositions
from torch.fx import GraphModule
import torch.nn.functional as F
import torch

class MyModule(torch.nn.Module):
    def forward(self, x, y):  # Accept two inputs
        # Ensure output is wrapped in a tuple
        return (F.gelu(x) + y,)  # Return as a tuple

mod = MyModule()

# Define example inputs as a tuple of two tensors
example_input = (
    torch.tensor([0.2, -0.5, 0.8, -1.2], requires_grad=True),
    torch.tensor([1, -1, 1, -1], dtype=torch.float32)
)

def print_backend(gm: GraphModule, inputs):
    print("Decomposed Graph:")
    print(gm.code)  # This will show decomposed ops like mul, erf, etc.
    return gm.forward

compiled = aot_module_simplified(
    mod,
    example_input,
    fw_compiler=print_backend,
    decompositions=core_aten_decompositions(),
)

# Call the compiled function with the two inputs
compiled(*example_input)

: 

In [2]:
from torch._functorch.aot_autograd import aot_module_simplified
from torch._decomp import core_aten_decompositions

from torch.fx import GraphModule
import torch.nn.functional as F
import torch

class MyModule(torch.nn.Module):
    def forward(self, x):
        return F.gelu(x)

mod = MyModule()
#example_input = (torch.randn(4),)
example_input = (torch.tensor([0.2, -0.5, 0.8, -1.2], requires_grad=True),torch.tensor([1, -1, 1, -1], dtype=torch.float32),)

def print_backend(gm: GraphModule, inputs):
    print("Decomposed Graph:")
    print(gm.code)  # <- This will show decomposed ops like mul, erf, etc.
    return gm.forward

compiled = aot_module_simplified(
    mod,
    example_input,
    fw_compiler=print_backend,
    decompositions=core_aten_decompositions(),
)

compiled(*example_input)

TypeError: MyModule.forward() takes 2 positional arguments but 3 were given

In [ ]:
# THIS IS THE SECOND ONE, IT WORKS BUT IT IS A BIT DIFFICULT TO MODIFY IT FOR OTHER OPERATORS

In [3]:
import torch
import torch.nn.functional as F
from torch._functorch.aot_autograd import aot_module_simplified
from torch._decomp import core_aten_decompositions
from torch.fx import GraphModule

# Define a custom module that uses soft_margin_loss
class MyModule(torch.nn.Module):
    def forward(self, *inputs):
        logits = inputs[0]
        targets = inputs[1]
        return (F.soft_margin_loss(logits, targets),)  # Use Soft Margin Loss in the forward pass

# Function to compute the loss (involving soft margin loss)
def compute_loss(logits, targets):
    # Compute soft margin loss
    loss = F.soft_margin_loss(logits, targets)
    return loss

# Example input tensors for logits and targets
logits = torch.randn(4, requires_grad=True)
targets = torch.tensor([1, -1, 1, -1], dtype=torch.float32)  # Targets must be -1 or 1

# Define a custom backend to print the decomposed graph
def print_backend(gm: GraphModule, inputs):
    print("Decomposed Graph:")
    print(gm.code)  # This will print the decomposed graph code
    return gm.forward  # Return the forward function for execution

# Use aot_module_simplified to compile the module and apply decompositions
compiled = aot_module_simplified(
    MyModule(),
    (logits, targets),
    fw_compiler=print_backend,  # Custom backend to print the graph
    decompositions=core_aten_decompositions(),  # Apply standard decompositions (including soft_margin_loss)
)

# Compute the loss by applying the forward pass (which triggers soft_margin_loss)
compute_loss(logits, targets)

Decomposed Graph:



def forward(self, primals_1, primals_2):
    neg = torch.ops.aten.neg.default(primals_1)
    mul = torch.ops.aten.mul.Tensor(neg, primals_2);  neg = None
    exp = torch.ops.aten.exp.default(mul);  mul = None
    log1p = torch.ops.aten.log1p.default(exp);  exp = None
    mean = torch.ops.aten.mean.default(log1p);  log1p = None
    return (mean, primals_1, primals_2)
    


tensor(0.7598, grad_fn=<SoftMarginLossBackward0>)

In [ ]:
# THIS IS NOT WORKING

In [6]:
import torch
import torch.nn.functional as F
from torch._functorch.aot_autograd import aot_module_simplified
from torch._decomp import core_aten_decompositions
from torch.fx import GraphModule

# Function to print the decomposition of a given operator and its inputs
def print_operator_decomposition(operator_fn, *inputs):
    # Define a custom module that applies the operator
    class OperatorModule(torch.nn.Module):
        def forward(self, *inputs):
            # Apply the given operator and return the result as a tuple
            return (operator_fn(*inputs),)  # Return the result as a tuple
    
    # Use aot_module_simplified to compile the module and apply decompositions
    def print_backend(gm: GraphModule, inputs):
        print("Decomposed Graph:")
        # Print out the graph in a human-readable format
        for node in gm.graph.nodes:
            print(f"{node.name} = {node.op}({', '.join(map(str, node.args))})")
        return gm.forward  # Return the forward function for execution

    # Use aot_module_simplified to compile and trace the module
    compiled = aot_module_simplified(
        OperatorModule(),
        *inputs,  # Provide the inputs to the operator
        fw_compiler=print_backend,  # Custom backend to print the graph
        decompositions=core_aten_decompositions(),  # Apply standard decompositions
    )

# Example usage: Allow dynamic input for the operator and its inputs
if __name__ == "__main__":
    # Example 1: Soft Margin Loss
    logits = torch.randn(4, requires_grad=True)
    targets = torch.tensor([1, -1, 1, -1], dtype=torch.float32)  # Targets must be -1 or 1
    print("Decomposing Soft Margin Loss:")
    print_operator_decomposition(F.soft_margin_loss, logits, targets)
    
    # Example 2: ReLU
    x = torch.randn(4, requires_grad=True)
    print("\nDecomposing ReLU:")
    print_operator_decomposition(F.relu, x)
    
    # Example 3: Batch Normalization
    x_bn = torch.randn(4, 4, requires_grad=True)
    print("\nDecomposing BatchNorm:")
    print_operator_decomposition(F.batch_norm, x_bn, running_mean=torch.zeros_like(x_bn), running_var=torch.ones_like(x_bn))
    

Decomposing Soft Margin Loss:


TypeError: aot_module_simplified() got multiple values for argument 'fw_compiler'